In [1]:
import tensorflow as tf
import keras
import keras.backend as K
from keras import optimizers
from keras import initializers
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense,AveragePooling2D
from keras.datasets import cifar100
from keras.models import Model

Using TensorFlow backend.


In [24]:
#Load dataset
(X, y_c), (x_test, y_c_test) = cifar100.load_data(label_mode='coarse')
(X, y), (x_test, y_test) = cifar100.load_data(label_mode='fine')

y_c = keras.utils.to_categorical(y_c, num_classes = num_classes_c)
y = keras.utils.to_categorical(y, num_classes = num_classes_f)

num_classes_c=20
num_classes_f=100

In [37]:
#Shared layers
in_layer = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

#block 1
net = Conv2D(32,kernel_size=(3, 3), activation="relu")(in_layer)
net = MaxPooling2D(pool_size=(2, 2)) (net)
net = Conv2D(64,kernel_size=(3, 3), activation="relu")(net)
net = MaxPooling2D(pool_size=(2, 2))(net)

#coarse model
net_c = Flatten()(net)
net_c = Dropout(0.5)(net_c)
net_c = Dense(num_classes_c, activation="softmax")(net_c)


#fine model 

net_f = Flatten()(net)
net_f = Dropout(0.5)(net_f)
net_f = Dense(num_classes_f, activation="softmax")(net_f)

# Two additional 'inputs' for the labels
label_layer_1 = Input((20,))
label_layer_2 = Input((100,))

# Construct your custom loss as a tensor
#using categorical crossentropy
cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
loss = cce(net_c, label_layer_1) + 0.5*cce(net_f, label_layer_2)

model = Model(inputs=[in_layer,label_layer_1, label_layer_2],outputs=[net_c,net_f])

# Add loss to model
model.add_loss(loss)

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 30, 30, 32)   896         main_input[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_21 (MaxPooling2D) (None, 15, 15, 32)   0           conv2d_21[0][0]                  
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 13, 13, 64)   18496       max_pooling2d_21[0][0]           
____________________________________________________________________________________________

In [ ]:
sgd_coarse = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer= sgd_coarse, metrics=['accuracy'])
batch = 64
#train model
model.fit([X,y_c,y], batch_size=batch,epochs=10)